this notebook makes sure that I can generate exactly the same model using either old or new code.

In [1]:
import numpy as np
import torch
from torch.autograd import Variable
from tang_jcompneuro_legacy import cnn as cnn_legacy
from tang_jcompneuro.cnn import CNN
from tang_jcompneuro.configs.cnn_arch import arch_dict
from tang_jcompneuro.configs.cnn_init import init_dict

In [2]:
arch_config = arch_dict['legacy_1L']['12']
init_config = init_dict['legacy']

In [3]:
parameter_mapping = {
    'conv.conv0.weight': 'features.0.weight',
    'conv.conv0.bias': 'features.0.bias',
    'fc.fc.weight': 'classifier.0.weight',
    'fc.fc.bias': 'classifier.0.bias',
}

def print_and_save_parameters(model):
    parameter_dict = {}
    for x, y in model.named_parameters():
        parameter_dict[x] = y.data.cpu().numpy().copy()
    return parameter_dict

def check_parameters(params_new, params_old):
    assert len(params_new) == len(params_old) == len(parameter_mapping)
    for x, y in params_new.items():
        # print(f'check {x}', y.shape)
        assert np.array_equal(y, params_old[parameter_mapping[x]])
    
    
def check():    
    for seed in range(10):
        model_new = CNN(arch_config, init_config, seed=seed)
        model_old = cnn_legacy._get_init_net('baseline', None, {'seed': seed}, cuda=False)
        # then let's compare two models.
        params_new = print_and_save_parameters(model_new)
        params_old = print_and_save_parameters(model_old)
        check_parameters(params_new, params_old)
        model_new.eval()
        model_old.eval()
        
        rng_state = np.random.RandomState(seed=seed)
        # generate some inputs.
        input_test = Variable(torch.FloatTensor(rng_state.randn(100,1,20,20)))
        output_1 = model_new(input_test).data.cpu().numpy().copy()
        output_2 = model_old(input_test).data.cpu().numpy().copy()
        assert output_1.shape == output_2.shape == (100, 1)
        assert np.array_equal(output_1, output_2)

In [4]:
check()